To run predictions, you first need to place the trained model file (.pt format) into the current directory

In [ ]:
import chemprop
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
df = pd.read_csv('data/separate_data/GLP_dataset.csv')
train = df[df['dataset_type'] == 'train']
train = train.drop(columns=['dataset_type'])
val = df[df['dataset_type'] == 'validation']
val= val.drop(columns=['dataset_type'])
test = df[df['dataset_type'] == 'test']
test = test.drop(columns=['dataset_type'])
train.to_csv('train_data.csv', index=False)
val.to_csv('val_data.csv', index=False)
test.to_csv('test_data.csv', index=False)

In [3]:
df = pd.read_csv(f'test_data.csv')

In [4]:
seed = 42

In [ ]:
checkpoint_dir = f"test_checkpoints_reg_seed_{seed}"
checkpoint_path = f"{checkpoint_dir}/model.pt"
save_dir = f"test_checkpoints_reg_seed_{seed}"  
model_objects = chemprop.utils.load_checkpoint(checkpoint_path)

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda


In [ ]:
arguments = [
    '--test_path', 'test_data.csv',
    '--preds_path', 'test_preds_reg.csv',
    '--checkpoint_dir', save_dir
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


6301it [00:00, 328931.24it/s]
100%|██████████| 6301/6301 [00:00<00:00, 337586.66it/s]


Validating SMILES
Test size = 6,301
Predicting with an ensemble of 1 models


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda


100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


Saving predictions to test_preds_reg.csv
Elapsed time = 0:00:04


In [7]:
df[f'logP_pred_{seed}'] = [x[0] for x in preds]
df

,molecule,logp,logP_pred_42
0,BrC(Br)C(Br)Br,2.55,2.684836
1,BrCCCCCCCBr,4.47,4.473289
2,BrCCN(CCBr)c1ccccc1,3.84,3.437457
3,Brc1cc(Br)c(Br)c(Br)c1,5.55,5.403308
4,Brc1cc(Br)c(Br)s1,4.48,4.659176
...,...,...,...
6296,c1ccc2nonc2c1,1.69,1.328012
6297,c1cnc2cscc2c1,1.74,2.092787
6298,c1cncc(-c2ccnc(-c3cccnc3)c2)c1,1.96,1.924181
6299,c1cncc(CCN2CCCCC2)c1,1.66,2.035929


In [8]:
r2 = r2_score(df['logp'], df[f'logP_pred_{seed}'])
rmse = np.sqrt(mean_squared_error(df['logp'], df[f'logP_pred_{seed}']))
mae = mean_absolute_error(df['logp'], df[f'logP_pred_{seed}'])
round(rmse, 3), round(r2, 3), round(mae, 3)

(0.521, 0.94, 0.317)

In [ ]:
df.to_csv('Seeds_D_MMPN_PREDICT_GLP_train_GLP_test.csv', index=False)